In [0]:
#mount an Azure Blob Storage container to Databricks File System (DBFS) so that you can access files stored in Azure as if they were local files in Databricks.
#dbutil - utility tool in databricks - enables to interact with databricks runtime environment 
#dbutil.fs - list files, copy/move/delete data
#mount() = attaches external storage to Databricks so you can use it like a folder
dbutils.fs.mount(
source = "wasbs://retail001@retailprojstorage.blob.core.windows.net",
mount_point = "/mnt/retail",
extra_options = {"fs.azure.account.key.retailprojstorage.blob.core.windows.net":"c+DN4Wyz+eKRpZkPxTOZIPF4ofjibotvC5b6PsDMHpSoMzKXeMPPR0OFqTLlXhXmsCg6hh7WQYoT+ASt/FEfoQ=="}
)
#wasbs:// is the protocol for Azure Blob
#mount point - databricks storage 
#extra_options - provide access keys to access the files in Azure

In [0]:
dbutils.fs.ls('/mnt/retail/bronze/transactions/')
#list all the files and folders located in the path

In [0]:
#read the raw data from bronze layer
df_customers = spark.read.json('/mnt/adls/retail/bronze/customers.json') 
df_products = spark.read.parquet('/mnt/adls/retail/bronze/products.csv')
df_stores = spark.read.parquet('/mnt/adls/retail/bronze/stores.csv')
df_transactions = spark.read.parquet('mnt/adls/retail/bronze/transactions.csv')
display(df_products)

In [0]:
display(df_transactions)
#show Dataframes or file listings

In [0]:
from pyspark.sql.functions import col

In [0]:
#convert types and clean data 
#transformations on transactions database (SQL DB)
df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"), 
    col("product_id").cast("int"), 
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

In [0]:
#transformation on products database (SQL DB)
df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

In [0]:
#transformation on stores database (SQL DB)
df_stores = df_stores.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

In [0]:
#transformation on customers data - json format data obtained by API call 
df_customers = df_customers.select(
    "customer_id","first_name","last_name","email","city","registration_date"
).dropDuplicates(["customer_id"])
#drop duplicate values and keep only unique values

In [0]:
#join all data 
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_stores, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))
#basically cleaning the data by joining all the dataframes

In [0]:
display(df_silver)
#show the dataframe

In [0]:
silver_path = "/mnt/adls/retail/silver/cleaned_transactions"
#setting path variable where the data will be saved - adls 
df_silver.write.mode("overwrite").parquet(silver_path)
#write tells spark to save the content and overwrite tells if there's already data at that path, delete it and write new data

In [0]:
spark.sql(f"""
    CREATE TABLE retail_silver_cleaned
    USING DELTA
    LOCATION '/mnt/retail/silver/';
          """)
#sql with spark - the data is stored in delta lake format

In [0]:
#Load cleaned transactions from Silver layer
silver_df = spark.read.format("delta").load("/mnt/adls/retail/silver/cleaned_transactions")

In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

In [0]:
gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id", "product_name","category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("avg_transactional_value")
)

In [0]:
gold_path = "/mnt/adls/retail/gold/"
df_gold.write.mode("overwrite").format("delta").save(gold_path)

In [0]:
spark.sql("""
    CREATE TABLE retail_gold_sales_summary
    USING DELTA
    LOCATION '/mnt/retail/gold/'
 """)